In [ ]:
# Import
import pandas as pd
import json
import textwrap
from datasets import load_dataset
import re
from transformers import pipeline

/home/aleksander/projects/DAB/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
ner = pipeline(task='ner', 
               model='saattrupdan/nbailab-base-ner-scandi', 
               aggregation_strategy='first')

result = ner('Opossums are great')

Device set to use cpu


In [14]:
with open("../annotation_experiments/pre_annotations_test.json", "r", encoding="utf-8") as doc1:
    anndoc = json.load(doc1)

In [17]:
# Sample JSON data
json_data = anndoc

# Sample NER output
ner_output = result

# Function to update JSON with NER predictions
def add_ner_predictions(json_data, ner_output, model_version="SkandiNER"):
    for item in json_data:
        text = item["data"]["text"]
        
        # Create a new predictions list if it doesn't exist
        if "predictions" not in item:
            item["predictions"] = []

        # Create a new result list
        results = []
        for i, entity in enumerate(ner_output, start=1):
            results.append({
                "id": str(i),
                "from_name": "entity_mentions",
                "to_name": "doc_text",
                "type": "labels",
                "value": {
                    "start": entity["start"],
                    "end": entity["end"],
                    "text": entity["word"],
                    "labels": [entity["entity_group"]]
                }
            })
        
        # Append the new model predictions
        item["predictions"].append({
            "model_version": model_version,
            "result": results
        })

    return json_data

# Update JSON structure with NER predictions
updated_json = add_ner_predictions(json_data, ner_output)

# Print the updated JSON
print(json.dumps(updated_json, indent=2))


[
  {
    "data": {
      "text": "Opossums are great",
      "source_dataset": "wiki"
    },
    "predictions": [
      {
        "model_version": "DaCy",
        "result": [
          {
            "id": "1",
            "from_name": "entity_mentions",
            "to_name": "doc_text",
            "type": "labels",
            "value": {
              "start": 0,
              "end": 9,
              "text": "Opossums",
              "labels": [
                "PERSON"
              ]
            }
          }
        ]
      },
      {
        "model_version": "SkandiNER",
        "result": [
          {
            "id": "1",
            "from_name": "entity_mentions",
            "to_name": "doc_text",
            "type": "labels",
            "value": {
              "start": 0,
              "end": 8,
              "text": "Opossums",
              "labels": [
                "ORG"
              ]
            }
          }
        ]
      }
    ]
  },
  {
    "data": {
      "

In [ ]:
import json

# JSON template
json_structure = [
    {
        "data": {
            "text": "",  # Placeholder
            "source_dataset": ""
        },
        "predictions": []  # Empty initially
    }
]

# Sample texts
texts = [
    {"text": "Opossums are great", "source_dataset": "wiki"},
    {"text": "And so is Sherlock Holmes", "source_dataset": "wiki"}
]

# Sample NER outputs
ner_outputs = {
    "Opossums are great": [
        {'entity_group': 'ANIMAL', 'score': 0.981, 'word': 'Opossums', 'start': 0, 'end': 8}
    ],
    "And so is Sherlock Holmes": [
        {'entity_group': 'PERSON', 'score': 0.974, 'word': 'Sherlock Holmes', 'start': 11, 'end': 26}
    ]
}

# Function to populate JSON
def populate_json(template, texts, ner_outputs, model_version="MyNERModel"):
    populated_json = []
    
    for text_entry in texts:
        new_entry = json.loads(json.dumps(template[0]))  # Deep copy template
        new_entry["data"]["text"] = text_entry["text"]
        new_entry["data"]["source_dataset"] = text_entry["source_dataset"]

        # Insert predictions if available
        text = text_entry["text"]
        if text in ner_outputs:
            results = []
            for i, entity in enumerate(ner_outputs[text], start=1):
                results.append({
                    "id": str(i),
                    "from_name": "entity_mentions",
                    "to_name": "doc_text",
                    "type": "labels",
                    "value": {
                        "start": entity["start"],
                        "end": entity["end"],
                        "text": entity["word"],
                        "labels": [entity["entity_group"]]
                    }
                })

            new_entry["predictions"].append({
                "model_version": model_version,
                "result": results
            })
        
        populated_json.append(new_entry)

    return populated_json

# Populate JSON structure with text and predictions
final_json = populate_json(json_structure, texts, ner_outputs)

# Print output
print(json.dumps(final_json, indent=2))
